In [ ]:
!pip install open3d --q
!pip install plotly --q
!pip install pyntcloud --q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 399.7/399.7 MB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 27.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.8/139.8 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 34.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 346.3/346.3 kB 10.8 MB/s eta 0:00:00


In [ ]:
import open3d as o3d
import numpy as np
import pandas as pd
import os
import re
from glob import glob
import time
import random
import seaborn as sns
import xml.etree.ElementTree as ET
from mpl_toolkits.mplot3d import Axes3D
from pyntcloud import PyntCloud
import plotly.graph_objs as go
from matplotlib.pyplot import cm
import matplotlib.pyplot as plt
from plotly.offline import iplot, init_notebook_mode

from sklearn.model_selection import train_test_split



import warnings
warnings.filterwarnings("ignore")

init_notebook_mode(connected=True)
%matplotlib inline

In [ ]:
ply_files = [
    "/content/data1.ply",
    "/content/data2.ply",
    "/content/data3.ply",
    "/content/data4.ply",
    "/content/data5.ply",
    "/content/data6.ply",
    "/content/data7.ply",
    "/content/data8.ply",
    "/content/data9.ply",
    "/content/data10.ply",
    "/content/data11.ply",
    "/content/data12.ply",
    "/content/data13.ply",
    "/content/data14.ply",
    "/content/data15.ply",
    "/content/data16.ply",
    "/content/data17.ply"
]

In [ ]:
!nvidia-smi

Fri Sep 27 07:01:39 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   37C    P8              12W /  70W |      3MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [ ]:
import open3d as o3d
import numpy as np



def extract_features_from_ply(ply_file):
    # Load PLY point cloud
    pcd = o3d.io.read_point_cloud(ply_file)
    points = np.asarray(pcd.points)

    # Calculate feature 1: Bounding box dimensions (x, y, z ranges)
    x_range = points[:, 0].max() - points[:, 0].min()
    y_range = points[:, 1].max() - points[:, 1].min()
    z_range = points[:, 2].max() - points[:, 2].min()

    # Calculate feature 2: Point density (points per volume unit)
    volume = x_range * y_range * z_range
    point_density = len(points) / volume

    # Calculate feature 3: Mean and Std deviation of height (z-axis)
    mean_z = np.mean(points[:, 2])
    std_z = np.std(points[:, 2])

    features = [x_range, y_range, z_range, point_density, mean_z, std_z]

    return np.array(features)

# Example usage
for ply_file in ply_files:
  features = extract_features_from_ply(ply_file)
  print("Extracted Features: ", features)

Extracted Features:  [116.02668381  52.00365448  10.45742667   0.42148626   0.32042109
   1.29925226]
Extracted Features:  [112.35417175  52.86379433   9.9542824    0.45104167   0.317899
   1.29288525]
Extracted Features:  [116.02530289  51.8754673   10.00501025   0.44618938   0.32636931
   1.29420382]
Extracted Features:  [116.03269958  52.9072876   10.47088587   0.41530473   0.33166028
   1.31043474]
Extracted Features:  [116.00077057  51.89946938  10.47235131   0.4242673    0.34378784
   1.32820419]
Extracted Features:  [112.25035477  52.8822937   10.45225012   0.43087981   0.32516133
   1.29520288]
Extracted Features:  [112.27480316  51.8783474   10.00501025   0.45816856   0.31204553
   1.28218067]
Extracted Features:  [84.77124786 51.79255486 10.01024389  0.61214779  0.31278946  1.28025494]
Extracted Features:  [115.9481163   52.8621521    9.97912848   0.44007379   0.30749902
   1.27372243]
Extracted Features:  [115.99201965  51.79454231  10.47088587   0.42793669   0.30651644
   1

In [ ]:
pip install pykalman

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.6/251.6 kB 14.4 MB/s eta 0:00:00


In [ ]:
import numpy as np
from sklearn import svm
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import precision_score, recall_score
from sklearn.cluster import DBSCAN
from pykalman import KalmanFilter
from sklearn.svm import SVC
import open3d as o3d

# Function to read and extract features from the PLY file
def read_ply(ply_file):
    try:
        pcd = o3d.io.read_point_cloud(ply_file)
        points = np.asarray(pcd.points)

        if points.size == 0:
            raise ValueError("Point cloud is empty")

        # Calculate feature 1: Bounding box dimensions (x, y, z ranges)
        x_range = points[:, 0].max() - points[:, 0].min()
        y_range = points[:, 1].max() - points[:, 1].min()
        z_range = points[:, 2].max() - points[:, 2].min()

        # Calculate feature 2: Point density (points per volume unit)
        volume = x_range * y_range * z_range
        point_density = len(points) / volume if volume > 0 else 0

        # Calculate feature 3: Mean and Std deviation of height (z-axis)
        mean_z = np.mean(points[:, 2])
        std_z = np.std(points[:, 2])

        ft_info = [x_range, y_range, z_range, point_density, mean_z, std_z]

        return points

    except Exception as e:
        print(f"Error reading PLY file: {e}")
        return None, None


# Remove ground points
def remove_ground_points(points,zmin):
    points[points[:, 2] >= zmin]
    pcd_from_points = o3d.geometry.PointCloud()
    pcd_from_points.points = o3d.utility.Vector3dVector(points)


    # Apply plane segmentation on the PointCloud
plane_model, inliers = pcd_from_points.segment_plane(distance_threshold=0.02,
                                                     ransac_n=3,
                                                     num_iterations=1000)

# Extract the planar inliers and outliers
inlier_cloud = pcd_from_points.select_by_index(inliers)  # Planar points
outlier_cloud = pcd_from_points.select_by_index(inliers, invert=True)  # Non-planar points

    return outlier_cloud


def extract_features(clusters):
    features = []
    for cluster in clusters:
        cluster_points = np.asarray(cluster.points)
        if cluster_points.shape[0] > 0:
            f1 = len(cluster_points)  # Number of points in the cluster
            f2 = np.min(np.linalg.norm(cluster_points, axis=1))  # Min distance from origin (sensor)
            cov_matrix = np.cov(cluster_points, rowvar=False)  # Covariance matrix
            f3 = cov_matrix.flatten()[:6]  # Simplified covariance features (6 elements)
            features.append(np.hstack([f1, f2, f3]))  # Concatenate features into a single array
    return np.array(features)

def train_svm(features, labels):
    if features.size == 0:
        raise ValueError("Features array is empty. Cannot train SVM.")

    # Ensure features are 2D
    if features.ndim == 1:
        features = features.reshape(-1, 1)

    scaler = StandardScaler()
    features_scaled = scaler.fit_transform(features)

    classifier = SVC()
    classifier.fit(features_scaled, labels)

    return classifier, scaler

# Kalman Filter for tracking
def kalman_tracking(points):
    kf = KalmanFilter(initial_state_mean=points[0], n_dim_obs=2)
    filtered_state_means, _ = kf.filter(points[:, :2])
    return filtered_state_means


In [ ]:
import open3d as o3d
from sklearn.cluster import DBSCAN

def hybrid_clustering(pcd, cluster_tolerance, min_cluster_size, max_cluster_size, distance_threshold, min_samples):
    points = np.asarray(pcd.points)  # Convert point cloud to numpy array

    kdtree = o3d.geometry.KDTreeFlann(pcd)  # PCL KDTree for efficient neighbor search
    kdtree.set_input_cloud(pcd)
    visited = np.zeros(len(points), dtype=bool)
    kdtree_clusters = []

    def cluster_grow(i, tolerance, visited, points, kdtree):
      neighbors = [i]
      cluster = []
      while neighbors:
          idx = neighbors.pop()
          if not visited[idx]:
              visited[idx] = True
              cluster.append(idx)
              # Using PCL's KDTree for faster radius search
              _, idx_neighbors = kdtree.radius_search_for_cloud(points[idx], tolerance)
              neighbors.extend([n for n in idx_neighbors if not visited[n]])
      return cluster


    # Iterate over all points and perform KDTree-based clustering
    for i in range(len(points)):
        if not visited[i]:
            cluster = cluster_grow(i, cluster_tolerance)
            if min_cluster_size < len(cluster) < max_cluster_size:
                kdtree_clusters.append(cluster)


    filtered_clusters = [cluster for cluster in kdtree_clusters if len(cluster) > min_cluster_size]

    for cluster in filtered_clusters:
        cluster_points = points[cluster]  # Extract points from this cluster
        dbscan = DBSCAN(eps=distance_threshold, min_samples=min_samples).fit(cluster_points)

        labels = dbscan.labels_
        unique_labels = np.unique(labels)

        # Step 4: Combine DBSCAN results into refined clusters
        refined_clusters = []
        for label in unique_labels:
            if label == -1:
                continue  # Skip noise points
            sub_cluster_indices = np.where(labels == label)[0]
            if min_cluster_size < len(sub_cluster_indices) < max_cluster_size:
                sub_cluster_points = cluster_points[sub_cluster_indices]
                refined_cluster = o3d.geometry.PointCloud()
                refined_cluster.points = o3d.utility.Vector3dVector(sub_cluster_points)
                refined_clusters.append(refined_cluster)

    return refined_clusters

In [ ]:
# Main pipeline
def main():
    # Step 1: Load LiDAR data
    for ply_file in ply_files:
      lidar_points = read_ply(ply_files[0])
      if lidar_points is None:
          return

    filtered_points = remove_ground_points(lidar_points, zmin=1.0)
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(filtered_points)  # Convert numpy array to Open3D PointCloud
    clusters = hybrid_clustering(pcd, cluster_tolerance=0.1,min_cluster_size=100,max_cluster_size=800, distance_threshold=0.3, min_samples=30)
    features = extract_features(clusters)

    #Step 5: Online learning - Initially supervised, then P-N experts for online updates
    initial_labels = np.random.choice([0, 1], size=features.shape[0])
    try:
        # Step 4: Train the SVM model
        classifier, scaler = train_svm(features, initial_labels)
        print("SVM model training complete")

    except ValueError as e:
        print(f"Error during SVM training: {e}")


    tracked_points = kalman_tracking(filtered_points)


    test_features = extract_features(clusters)
    test_scaled = scaler.transform(test_features)
    predictions = classifier.predict(test_scaled)

    # Step 8: Performance evaluation (precision, recall, etc.)
    precision = precision_score(initial_labels, predictions)
    recall = recall_score(initial_labels, predictions)
    print(f'Precision: {precision:.2f}, Recall: {recall:.2f}')

if __name__ == "__main__":
    main()

AttributeError: 'numpy.ndarray' object has no attribute 'segment_plane'

In [ ]:
print(clusters)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

def visualize_clusters(points, clusters):
    points = np.asarray(points)  # Convert to NumPy array if not already
    clusters = np.asarray(clusters)  # Convert clusters to NumPy array if not already

    unique_clusters = np.unique(clusters)
    colors = plt.cm.viridis(np.linspace(0, 1, len(unique_clusters)))  # Use a colormap

    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')

    for cluster_id, color in zip(unique_clusters, colors):
        cluster_points = points[clusters == cluster_id]
        if cluster_points.size > 0:  # Check if cluster_points has data
            ax.scatter(cluster_points[:, 0], cluster_points[:, 1], cluster_points[:, 2],
                       color=color, label=f'Cluster {cluster_id}', s=5)

    ax.set_xlabel('X Axis')
    ax.set_ylabel('Y Axis')
    ax.set_zlabel('Z Axis')
    ax.set_title('3D Clustering Visualization')
    ax.legend()
    plt.show()

# Example of how to call it after clustering
if clusters is not None and len(clusters) > 0:  # Check if clusters are available
    visualize_clusters(np.asarray(pcd.points), clusters)  # Pass the points and clusters
else:
    print("Clusters not found or empty. Please check your clustering implementation.")
